# Task 2

In [0]:
from pyspark.sql.functions import concat, col, lit 

In [0]:
from pyspark.sql.functions import col, unix_timestamp, to_date
import pyspark.sql.functions as f
dbfs_fileStore_prefix = "/FileStore/tables"
prefix = "ontimeperformance"
size = "small"
year = 2000

In [0]:
def clean_column_names(df):
  tempList = [] #Edit01
  for col in df.columns:
      new_name = col.strip()
      new_name = "".join(new_name.split())
      new_name = new_name.replace('.','') 
      tempList.append(new_name) 

  return df.toDF(*tempList) 


In [0]:
def task_2(spark_session, flights_path, airlines_path, year):
    flights_df = spark_session.read.csv(flights_path,
                                        inferSchema=True,
                                        header=True)
    flights_df = clean_column_names(flights_df).dropna()

    flights_df = flights_df.withColumn('dates',  to_date(unix_timestamp(col('flight_date'), 'yyyy-MM-dd').cast("timestamp")))
    flights_df = flights_df.withColumn('year',f.year(f.to_timestamp('dates', 'dd/MM/yyyy')))
    
    flights_df = flights_df.withColumn('delay_time', flights_df.actual_departure_time - flights_df.scheduled_depature_time)
    
    
    flights_df = flights_df.select([c for c in flights_df.columns if c in {'carrier_code','delay_time', 'year'}]).filter(flights_df.delay_time > 0)
    airlines_df = spark_session.read.csv(airlines_path,
                                        inferSchema=True,
                                        header=True)
    airlines_df = clean_column_names(airlines_df).dropna().filter(airlines_df.country == "United States")
    
    US_flights  = flights_df.join(airlines_df, flights_df.carrier_code == airlines_df.carrier_code, how='inner')
    

    US_flights = US_flights.filter(US_flights.year == year)
    US_flights = US_flights.drop('year')
    delay_count = US_flights.groupBy(US_flights.name).count()
    avg_delay = US_flights.groupBy(US_flights.name).avg()
    min_delay = US_flights.groupBy(US_flights.name).min()
    max_delay = US_flights.groupBy(US_flights.name).max()
    
    join_table = delay_count.join(avg_delay, on = 'name').join(min_delay, on = 'name').join(max_delay, on = 'name')
    result = join_table.toDF('airline_name', 'num_delays', 'average_delay', 'min_delay', 'max_delay')
    
    result = result.orderBy('airline_name')
    
    result = result.select("*", concat(col("airline_name"),lit(' \t '),col("num_delays"),lit(' \t '), col("average_delay"), col("min_delay"),lit(' \t '), col("max_delay")).alias("Result"))
    
    
    # SAVE THE RESULT COLUMN TO CSV 
    result.select("Result").write.mode("overwrite").csv("dbfs:/FileStore/tables/task2")
    
    

#     print(result)
    
#     for i in result:
#       print(i[0])
    
#     haven't output as a file



In [0]:
task_2(spark, f"{dbfs_fileStore_prefix}/{prefix}_flights_{size}.csv", f"{dbfs_fileStore_prefix}/{prefix}_airlines.csv", year)

In [0]:
data_location = "dbfs:/FileStore/tables/task2"

files = dbutils.fs.ls(data_location)
csv_file = [x.path for x in files if x.path.endswith(".csv")][0]
dbutils.fs.mv(csv_file, data_location.rstrip('/') + ".csv")
dbutils.fs.rm(data_location, recurse = True)

Out[8]: True

# See the csv file we stored from a folder

In [0]:
dbutils.fs.ls("dbfs:/FileStore/tables")

Out[9]: [FileInfo(path='dbfs:/FileStore/tables/comments-1.csv', name='comments-1.csv', size=28057586),
 FileInfo(path='dbfs:/FileStore/tables/comments.csv', name='comments.csv', size=28057586),
 FileInfo(path='dbfs:/FileStore/tables/hamlet.txt', name='hamlet.txt', size=210705),
 FileInfo(path='dbfs:/FileStore/tables/ontimeperformance_aircrafts.csv', name='ontimeperformance_aircrafts.csv', size=428796),
 FileInfo(path='dbfs:/FileStore/tables/ontimeperformance_airlines.csv', name='ontimeperformance_airlines.csv', size=19691),
 FileInfo(path='dbfs:/FileStore/tables/ontimeperformance_airports.csv', name='ontimeperformance_airports.csv', size=244438),
 FileInfo(path='dbfs:/FileStore/tables/ontimeperformance_flights_large.csv', name='ontimeperformance_flights_large.csv', size=1050293591),
 FileInfo(path='dbfs:/FileStore/tables/ontimeperformance_flights_medium.csv', name='ontimeperformance_flights_medium.csv', size=105063706),
 FileInfo(path='dbfs:/FileStore/tables/ontimeperformance_flights_small.csv', name='ontimeperformance_flights_small.csv', size=10524263),
 FileInfo(path='dbfs:/FileStore/tables/task2.csv', name='task2.csv', size=413),
 FileInfo(path='dbfs:/FileStore/tables/task2.tsv/', name='task2.tsv/', size=0)]

# Read the csv file we stored

In [0]:
resCsv = sqlContext.read.format("csv").load("/FileStore/tables/task2.csv")
resCsv.show()

+--------------------+
 _c0|
+--------------------+
Alaska Airlines I...|
American Airlines...|
Continental Air L...|
Delta Air Lines I...|
Northwest Airline...|
Southwest Airline...|
US Airways 233 ...|
United Airlines ...|
+--------------------+

#Print the concent of the results out

In [0]:
resCsv.collect()

Out[11]: [Row(_c0='Alaska Airlines Inc. \t 64 \t 43.8751 \t 199'),
 Row(_c0='American Airlines Inc. \t 250 \t 55.4281 \t 1946'),
 Row(_c0='Continental Air Lines Inc. \t 124 \t 33.5967741935483841 \t 376'),
 Row(_c0='Delta Air Lines Inc. \t 385 \t 41.948051948051951 \t 2345'),
 Row(_c0='Northwest Airlines Inc. \t 144 \t 53.2291666666666641 \t 397'),
 Row(_c0='Southwest Airlines Co. \t 357 \t 48.7198879551820742 \t 333'),
 Row(_c0='US Airways \t 233 \t 40.729613733905581 \t 574'),
 Row(_c0='United Airlines \t 372 \t 56.048387096774191 \t 1506')]